## Configuration setup
### TODO: remove unused functions, imports, variables and commented out cells
### This requires a config.py file to run
### Specify a treatment name among the options ("alternatives" & "second_line"), default is "alternatives"

In [ ]:
#specify treatment
# treatment_name = "alternatives"
treatment_name = "second_line"
in_name = "manuscript_covariates_5_final"

treatment = treatment_name.capitalize()


In [ ]:
#specifying input and output of the function
# in_name_1 = "manuscript_covariates_final"
in_name_2 = f"gen_uti_cohort-first-{treatment_name}"
in_name_3 = f"cache_data_uti_omop_first_{treatment_name}"

# out_name_1 = f"cohort_features_{treatment}_adverse_events_censor_v0.pkl"
# out_name_2 = f"omop_1_year_features_{treatment}_adverse_events_censor_v0.pkl"
# out_name_3 = f"cohort_features_{treatment}_treatment efficacy_censor_v0.pkl"
# out_name_4 = f"omop_1_year_features_{treatment}_treatment efficacy_censor_v0.pkl"

#out_name_5... = all the grid search files in Logs/Grid Search

## Loading Datasets

In [ ]:
import psycopg2
import pandas as pd
import numpy as np

from collections import defaultdict
from sklearn.utils import resample
import sys
import time
import importlib
import sparse
import datetime
import scipy.sparse
from scipy.sparse import vstack
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import lightgbm as lgb
import pickle

from __future__ import division
from tqdm import tqdm

import gc
import warnings
warnings.filterwarnings('ignore')

import Utils.dbutils as dbutils
import Utils.data_utils as data_utils
import Generators.CohortGenerator as CohortGenerator
import Generators.FeatureGenerator as FeatureGenerator
import config
import copy
local_imports = (
    dbutils,
    data_utils,
    CohortGenerator,
    FeatureGenerator,
    config
)
for i in local_imports:
    i = importlib.reload(i)



database_name = 'omop_v6'
conn_string = "dbname="+database_name + " host=/var/run/postgresql"
conn=psycopg2.connect(conn_string)
print('Connected!')
database_name = config.DB_NAME
print(database_name)
config_path = 'postgresql://{database_name}'.format(
    database_name = database_name
)
connect_args = {"host": '/var/run/postgresql/'} # connect_args to pass to sqlalchemy create_engine function

# schemas 
schema_name = 'cdm_6871_21' # all created tables will be created using this schema
cdm_schema_name = config.OMOP_CDM_SCHEMA # the name of the schema housing your OMOP CDM tables
print(f"cdm schema: {cdm_schema_name}")
# set up database, reset schemas as needed
db = dbutils.Database(config_path, schema_name, connect_args, cdm_schema_name)

In [ ]:
%%time
new_cohort = pd.read_sql(f"SELECT * FROM cdm_6871_21.{in_name}", conn)
new_cohort = new_cohort.loc[new_cohort.antibiotic_type.isin(['nitrofurantoin','trimethoprim-sulfamethoxazole',treatment_name])]
#initialize feature dictionary
feat_dict = defaultdict(dict)

In [ ]:
print(in_name)

In [ ]:
new_cohort.antibiotic_type.value_counts()

# Only run if using OMOP (optional)

## Specifying cohort and feature config

In [ ]:
omop_name = f'__uti_cohort_omop_first_{treatment_name}'
cohort_script_path = config.SQL_PATH_COHORTS + f'/gen_uti_cohort-first-{treatment_name}.sql'

# cohort parameters  
params = {
          'cohort_table_name'     : omop_name,
          'schema_name'           : schema_name,
          'aux_data_schema'       : config.CDM_AUX_SCHEMA,
          'training_start_date'   : '2009-01-01',
          'dummy_date'            : '1900-01-01'
         }

cohort = CohortGenerator.Cohort(
    schema_name=schema_name,
    cohort_table_name=omop_name,
    cohort_generation_script=cohort_script_path,
    cohort_generation_kwargs=params,
    outcome_col_name='y'
)

featureSet = FeatureGenerator.FeatureSet(db)
featureSet.add_default_features(
    ['drugs_relative','conditions_relative','procedures_relative','specialty_relative'],
    schema_name,
    omop_name
)

## Building cohort and omop features.  I have already built the table and the features so you can use replace=False flag, and from_cached=True to retrieve them assuming access to the  cache_data_path and cohort_script_path

In [ ]:
%%time
cohort.build(db, replace=False)

In [ ]:
%%time
# Build the Feature Set by executing SQL queries and reading into sparse matrices
cache_data_path = f'/data/ibc_6871_21/ncjones_ibc/cache_data_uti_omop_first_{treatment_name}'
featureSet.build(cohort, from_cached=True, cache_file=cache_data_path)

##  Loading Feature matrices

In [ ]:
outcomes_filt, feature_matrix_3d_transpose, remap, good_feature_names = \
    FeatureGenerator.postprocess_feature_matrix(cohort, featureSet,training_end_date_col='dummy_date')

### OMOP Features

In [ ]:
%%time
#1 year

feature_matrix_counts_1, feature_names_1 = data_utils.window_data(
    window_lengths = [3,7,30,180,365],
    feature_matrix = feature_matrix_3d_transpose,
    all_feature_names = good_feature_names,
    cohort = cohort,
    featureSet = featureSet,
    cohort_end_date_col='dummy_date'
)


### Add antibiotic feature to omop features

In [ ]:
%%time 

#filtering for feature columns
feature_dict = featureSet.__dict__['id_map_rev']
feature_set = list(feature_dict.values())
ids_to_exclude = [x for x in feature_set if x not in remap['id']]
new_feature_dict = {k:v for k,v in feature_dict.items() if v not in ids_to_exclude}


new_cohort = new_cohort.loc[new_cohort.condition_occurrence_id.isin(list(new_feature_dict.keys()))]
new_cohort = new_cohort.set_index('condition_occurrence_id')
new_cohort = new_cohort.reindex(list(new_feature_dict.keys()))
new_cohort = new_cohort.reset_index()

filtered_cohort = new_cohort.copy()
filtered_cohort['treatment_'] = filtered_cohort.antibiotic_type == treatment_name

#combining features

combined_features_1 = vstack([feature_matrix_counts_1, scipy.sparse.csr_matrix(filtered_cohort['treatment_'])]).T
combined_feature_names_1 = feature_names_1 + ['treatment_']


#update feat_dict
feat_dict['treatment']["omop_1_year_features"] = (feature_matrix_counts_1.T, feature_names_1)
feat_dict['censor']["omop_1_year_features"] = (combined_features_1, combined_feature_names_1)





# End of OMOP Portion (currently not optional because the dataframe size of new cohort changes when adding negatives)

In [ ]:
%%time 

filtered_cohort = new_cohort.copy()


cols_to_exclude = [
'no_previous_180_excluded_event',
'no_previous_180_day_event',
'no_two_previous_365_day_event',
 'no_previous_excluded_event_ever',
'previous_uti_condition_occurence_id',
 'multi',
 'year_of_birth',
'level_0',
 'index',
 'condition_occurrence_id',
 'person_id',
 'condition_concept_id',
 'condition_start_date',
 'condition_start_datetime',
 'condition_end_date',
 'condition_end_datetime',
 'visit_occurrence_id',
 'visit_detail_id',
 'drug_concept_id',
 'drug_name',
 'antibiotic_name',
 'antibiotic_type',
 'visit_provider_id',
 'drug_exposure_id',
 'drug_exposure_start_date',
 'drug_exposure_start_datetime',
 'provider_id',
 'provider_name',
 'npi',
 'post_UTI_codes',
 'recurrent_uti',
 'first_uti',
 'followup_time',
 't_sum',
 't_bin',
 't_uti_bin',
 't_neph_bin',
 'AE_c_diff',
 'AE_GI',
 'AE_skin',
 'AE_AKI',
 'AE_other',
 't_sepsis_sum',
 't_i_sepsis_sum',
 't_i_uti_sum',
 't_i_neph_sum',
 't_sepsis_bin',
 't_i_uti_bin',
 't_i_neph_bin',
 't_i_sepsis_bin',
 't_i_sum',
 't_uti_sum',
 't_neph_sum',
 't_i_bin','AE_any','less_15','less_30','less_90','followup_time'] + \
 [x for x in filtered_cohort.columns if 'full_condition_name' in x] + \
 ['fibro_' + m + '_mon_outcome' for m in ['1','3','6']] + \
 ['hernia_' + m + '_mon_outcome' for m in ['1','3','6']] + \
 ['fracture_' + m + '_mon_outcome' for m in ['1','3','6']]
cohort_features = filtered_cohort.drop(columns=cols_to_exclude)


## Create outcome and feature dictionary

In [ ]:
outcome_dict = {'treatment':{'first_line':(1*(filtered_cohort.antibiotic_type=="nitrofurantoin") + 1*(filtered_cohort.antibiotic_type=="trimethoprim-sulfamethoxazole")),
                             'second_line':1*filtered_cohort.antibiotic_type=='second_line',
                             'alternatives':1*filtered_cohort.antibiotic_type=='alternatives'},
                'censor' : {'15':1*filtered_cohort.less_15==0,
                                 '30':1*filtered_cohort.less_30==0,
                                 '90':1*filtered_cohort.less_90==0}
                    
}


mod_cohort_features = cohort_features.copy()
mod_cohort_features['treatment_'] = 1*(filtered_cohort.antibiotic_type==treatment_name)


#update feature dictionary
feat_dict['treatment']["cohort_features"] = (scipy.sparse.csr_matrix(cohort_features.values), list(cohort_features.columns))
feat_dict['censor']["cohort_features"] =  (scipy.sparse.csr_matrix(mod_cohort_features.values), list(mod_cohort_features.columns))
              


## Show Variable Shapes

In [ ]:
print("Treatment variable shapes")
print("features")
for key, value in feat_dict["treatment"].items():
    print(key, value[0].shape)
print("outcomes")
for key, value in outcome_dict["treatment"].items():
    print(key, value.shape)
    
    
print("\n\nCensor variable shapes")
print("features")
for key, value in feat_dict["censor"].items():
    print(key, value[0].shape)

print("outcomes")
for key, value in outcome_dict["censor"].items():
    print(key, value.shape)


### Checking the variable shapes

# Analysis

In [ ]:
##Grid Search and plotting Helper Functions
def run_logreg(matrix : scipy.sparse.csr.csr_matrix, y, CV=False,folds=3,params=None,search=False,add_propensity=False):
    sparse_features = scipy.sparse.csr_matrix(matrix)
    X, T = sparse_features, y.values
    # build model for p(t=1|x); prob of getting first-line antibiotic 
    X_Train, X_Test, T_Train, T_Test = train_test_split(X, T, test_size = 0.2, random_state = 0)

    X_Dev, X_Val, T_Dev, T_Val = train_test_split(X_Train, T_Train, test_size=0.25, random_state=0)
    
    scaler = MaxAbsScaler()
    X_Train = scaler.fit_transform(X_Train)
    X_Test = scaler.transform(X_Test)
    
    X_Dev = scaler.fit_transform(X_Dev)
    X_Val = scaler.transform(X_Val)

    if search:
        C=[.01,.1,.25,.5,1]
        penalty=['l1','l2']
        params_names = ["C","penalty"]

        if not CV:
            best_params, best_score = [0.1, 'l1'], float('-inf')
            for c in C:
                for p in penalty:
                    model = LogisticRegression(random_state=0, penalty=p, C=c, solver='liblinear')
                    model.fit(X_Dev, T_Dev)
                    s = roc_auc_score(T_Val, model.predict_proba(X_Val)[:,1])
                    # print(c,p,s)
                    if s > best_score:
                        best_params, best_score = [c,p], s

            classifier = LogisticRegression(random_state=0, penalty=best_params[1], C=best_params[0], solver='liblinear')
            best_params = dict(zip(params_names,best_params))
        else:
            clf = GridSearchCV(LogisticRegression(C=0.25, penalty='l1',solver='liblinear', random_state=0,verbose=0),
                               param_grid={'C':C,'penalty':penalty},cv=folds,scoring="roc_auc",refit=True
                         )
            clf.fit(X_Train,T_Train)
            classifier = clf.best_estimator_
            best_params = clf.best_params_
            best_score = clf.best_score_
    else:
        classifier = LogisticRegression(random_state=0, penalty=params['penalty'], C=params['C'], solver=params['solver'])
        best_params = params
        best_score = None
    classifier.fit(X_Train, T_Train)

    T_Pred = classifier.predict(X_Test)
    T_Pred_Prob = classifier.predict_proba(X_Test)
#     print("Accuracy:", np.round(metrics.accuracy_score(T_Test, T_Pred), 4))


    acc = np.round(metrics.accuracy_score(T_Test, T_Pred), 4)
    fpr, tpr, threshold = (metrics.roc_curve(T_Test, T_Pred_Prob[:,1]))
    roc_auc = metrics.auc(fpr, tpr)
    
    if add_propensity:
        cohort = filtered_cohort.copy()
        classifier.fit(X, T)
        cohort['propensity_score'] = np.clip(classifier.predict_proba(X)[:,1], 0.025, 0.975)
        cohort['treatment'] = y
        return cohort, classifier, best_params, best_score, (acc,roc_auc,fpr, tpr)
    else:
        return classifier

from sklearn.ensemble import RandomForestClassifier



#modified for omop
def run_rf(matrix, y, CV=False,folds=3,search=False,params=None,add_propensity=False):
    sparse_features = scipy.sparse.csr_matrix(matrix)
    X, T = sparse_features, y.values
    # build model for p(t=1|x); prob of getting first-line antibiotic 
    X_Train, X_Test, T_Train, T_Test = train_test_split(X, T, test_size = 0.2, random_state = 0)

    X_Dev, X_Val, T_Dev, T_Val = train_test_split(X_Train, T_Train, test_size=0.25, random_state=0)
    
    scaler = MaxAbsScaler()
    X_Train = scaler.fit_transform(X_Train)
    X_Test = scaler.transform(X_Test)
    
    X_Dev = scaler.fit_transform(X_Dev)
    X_Val = scaler.transform(X_Val)

    max_features=['auto', 'sqrt', 'log2']
    n_estimators=[60, 80, 100, 120, 140]
    min_samples_leaf=[25, 50, 75]
    params_names = ["max_features","n_estimators","min_samples"]
    
    
    if search:
        if not CV:
            best_params, best_score = [max_features[0], n_estimators[0], min_samples_leaf[0]], float('-inf')

            for f in max_features:
                for e in n_estimators:
                    for l in min_samples_leaf:
                        model=RandomForestClassifier(random_state=0, n_estimators = e, max_features = f, min_samples_leaf = l)
                        model.fit(X_Dev, T_Dev)
                        s = roc_auc_score(T_Val, model.predict_proba(X_Val)[:,1])
                        # print(c,p,s)
                        if s > best_score:
                            best_params, best_score = [f, e, l], s


            # #Train the model using the training sets y_pred=model.predict(X_test)
            classifier = RandomForestClassifier(random_state=0, max_features=best_params[0],n_estimators=best_params[1],min_samples_leaf=best_params[2])
            best_params = dict(zip(params_names,best_params))
        else:
            clf = GridSearchCV(RandomForestClassifier(n_estimators=100, max_features="auto",min_samples_leaf=50, random_state=0),
                               param_grid={'max_features':max_features,'n_estimators':n_estimators, "min_samples_leaf":min_samples_leaf},cv=folds,scoring="roc_auc",refit=True
                         )
            clf.fit(X_Train,T_Train)
            classifier = clf.best_estimator_
            best_params = clf.best_params_
            best_score = clf.best_score_
    else:
        classifier = RandomForestClassifier(random_state=0, n_estimators=params['n_estimators'], max_features=params['max_features'], min_samples_leaf=params['min_samples_leaf'])
        best_params = params
        best_score = None

    classifier.fit(X_Train, T_Train)

    T_Pred = classifier.predict(X_Test)
    T_Pred_Prob = classifier.predict_proba(X_Test)
#     print("Accuracy:", np.round(metrics.accuracy_score(T_Test, T_Pred), 4))

    acc = np.round(metrics.accuracy_score(T_Test, T_Pred), 4)
    fpr, tpr, threshold = (metrics.roc_curve(T_Test, T_Pred_Prob[:,1]))
    roc_auc = metrics.auc(fpr, tpr)

    
    if add_propensity:
        cohort = filtered_cohort.copy()
        classifier.fit(X, T)
        cohort['propensity_score'] = np.clip(classifier.predict_proba(X)[:,1], 0.025, 0.975)
        cohort['treatment'] = y
        return cohort, classifier, best_params, best_score, (acc,roc_auc,fpr, tpr)
    else:
        return classifier

def run_lgbm(matrix, y, add_propensity=False,CV=False,folds=3,search=True,params=None):
    sparse_features = scipy.sparse.csr_matrix(matrix)
    X, T = sparse_features, y.values
    # build model for p(t=1|x); prob of getting first-line antibiotic 

    X_Train, X_Test, T_Train, T_Test = train_test_split(X, T, test_size = 0.2, random_state = 0)

    X_Dev, X_Val, T_Dev, T_Val = train_test_split(X_Train, T_Train, test_size=0.25, random_state=0)
    
    scaler = MaxAbsScaler()
    X_Train = scaler.fit_transform(X_Train)
    X_Test = scaler.transform(X_Test)
    
    X_Dev = scaler.fit_transform(X_Dev)
    X_Val = scaler.transform(X_Val)

    boosting_type=["gbdt","dart"]
    if not search:
        learning_rates=[ .05]
        num_leaves=[50]
        if params:
            lr = params["learning_rate"]
            num_leaves = params["num_leaves"]
            classifier = lgb.LGBMClassifier(random_state=0,learning_rate=lr,num_leaves=num_leaves)
            best_params = params
            best_score = None

    else:
        learning_rates=[.001,.005, .01, .05, .1]
        num_leaves=[10,50,100,250]
        params_names = ["learning_rate","num_leaves"]
    
        if not CV:
            best_params, best_score = [learning_rates[0], num_leaves[0]], float('-inf')

            for l in learning_rates:
                for n in num_leaves:
                    model = lgb.LGBMClassifier(num_leaves=n, learning_rate=l,random_state=0)
                    model.fit(X_Dev, T_Dev)
                    s = roc_auc_score(T_Val, model.predict_proba(X_Val)[:,1])
                    # print(c,p,s)
                    if s > best_score:
                        best_params, best_score = [l, n], s


            # #Train the model using the training sets y_pred=model.predict(X_test)
            classifier = lgb.LGBMClassifier(random_state=0,learning_rate=best_params[0],num_leaves=best_params[1])
            best_params = dict(zip(params_names,best_params))
            best_params["boosting_type"] = 'gbdt'
        else:
            clf = GridSearchCV(lgb.LGBMClassifier(boosting_type="gbdt", learning_rate=.01,num_leaves=50, random_state=0),
                               param_grid={'boosting_type':boosting_type,'learning_rate':learning_rates, "num_leaves":num_leaves},cv=folds,scoring="roc_auc",refit=True
                         )
            clf.fit(X_Train,T_Train)
            classifier = clf.best_estimator_
            best_params = clf.best_params_
            best_score = clf.best_score_        

    classifier.fit(X_Train, T_Train)

    T_Pred = classifier.predict(X_Test)
    T_Pred_Prob = classifier.predict_proba(X_Test)
#     print("Accuracy:", np.round(metrics.accuracy_score(T_Test, T_Pred), 4))


    acc = np.round(metrics.accuracy_score(T_Test, T_Pred), 4)
    fpr, tpr, threshold = (metrics.roc_curve(T_Test, T_Pred_Prob[:,1]))
    roc_auc = metrics.auc(fpr, tpr)


    if add_propensity:
        cohort = filtered_cohort.copy()
        classifier.fit(X, T)
        cohort['propensity_score'] = np.clip(classifier.predict_proba(X)[:,1], 0.025, 0.975)
        cohort['treatment'] = y
#         param_dictionary = dict(zip(params_names,best_params))
#         param_dictionary["boosting_type"] = "gbdt"


        return cohort, classifier, best_params, best_score, (acc,roc_auc,fpr, tpr)
    else:
        return classifier
    
def compute_roc_auc(auc_metrics, name=None, show=None):
    roc_auc, fpr, tpr = auc_metrics
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    if name:
        plt.savefig(name)
    if not show:
        plt.close()

def build_model(feature,outcome,y,model,cv=False,folds=3):
    #assumes the functions can actually generate these things
    if model == 1:
        cohort, clf, params, scores, metric = run_logreg(feat_dict[outcome][feature][0],y,CV=cv,folds=folds,add_propensity=True)
        return cohort, clf, params, scores, metric
    elif model == 2:
        cohort, clf, params, scores, metric = run_rf(feat_dict[outcome][feature][0],y,CV=cv,folds=folds,add_propensity=True)
        return cohort, clf, params, scores, metric
    elif model == 3:
        cohort, clf, params, scores, metric = run_lgbm(feat_dict[outcome][feature][0],y,CV=cv,folds=folds,add_propensity=True)
        return cohort, clf, params, scores, metric
        
def reweighting(cohort,name=None,show=False):
    '''
    Input: cohort with a propensity column named propensity_score and antibiotic type column
           treatment name
    
    return plot'''
    first_line, non_first_line = cohort[cohort['treatment']==1], cohort[cohort['treatment']==0]
    n = cohort.shape[0]

    age_range = np.arange(int(cohort.age.min()), int(cohort.age.max()+1))
    counts1, counts0 = [], []
    for a in age_range:
        counts1.append(sum(1/first_line[first_line.age==a].propensity_score))
        counts0.append(sum(1/(1-non_first_line[non_first_line.age==a].propensity_score)))


    fig, axs = plt.subplots(1,2, figsize=(12, 5))
    # plt.rcParams["figure.figsize"] = [7.50, 5.50]
    # plt.rcParams["figure.autolayout"] = True
    axs[0].fill_between(age_range, counts1, step="pre", alpha=0.5, label='First-line')
    axs[0].fill_between(age_range, counts0, step="pre", alpha=0.5, label=treatment)

    # plt.plot(range(18,100),counts1, drawstyle="steps")
    # plt.plot(range(18,100),counts0, drawstyle="steps")
    axs[0].set_title("Age distribution reweighted by propensity scores", size=13)

    axs[1].hist(first_line.age, bins=88, alpha=0.5, label="First-line");
    axs[1].hist(non_first_line.age, bins=88, alpha=0.5, label=treatment)
    axs[1].set_title("Age distribution before reweighting", size=13)

    for ax in axs:
        ax.set_xlabel("Age", size=11)
        ax.set_ylabel("Count", size=11)
        ax.legend()

    avg1 = np.average(age_range, weights=counts1)
    avg0 = np.average(age_range, weights=counts0)
    avg_all = np.average(age_range, weights=np.array(counts1)+np.array(counts0))
    sigma = np.sqrt(np.average((age_range-avg_all)**2, weights=np.array(counts1)+np.array(counts0)))
    age_diff = (avg1-avg0)/sigma
    
    if name:
        plt.savefig(name)
    if not show:
        plt.close()
    return fig, age_diff
    
     
def propensity(cohort,name=None,show=False):
    '''
    Input: cohort with a propensity column named propensity_score and antibiotic type column
           treatment name
    
    return plot'''

    plt.figure(figsize=(8,6))
    t1 = cohort[cohort['treatment']==1].propensity_score
    t2 = cohort[cohort['treatment']==0].propensity_score
    weight1 = np.ones_like(t1) / len(t1)
    weight2 = np.ones_like(t2) / len(t2)

    plt.hist(t1, weights=weight1, bins=50, alpha=0.5, label="First line (T=1)")
    plt.hist(t2, weights=weight2, bins=50, alpha=0.5, label=f"{treatment} (T=0)")

    plt.xlabel("Propensity score", size=14)
    plt.ylabel("Count", size=14)
    plt.title(f"Distribution of Propensity Scores (First line vs {treatment})")
    plt.legend(loc='upper right')
    # plt.savefig("overlapping_histograms_with_matplotlib_Python.png")
    if name:
        plt.savefig(name)
    if not show:
        plt.close()
    return plt
    
def calibration(cohort,name=None,show=False):
    '''
    Input: cohort with a propensity column named propensity_score and antibiotic type column
           treatment name
    
    return plot'''
   
    bins = [np.quantile(cohort.propensity_score, x/8) for x in range(9)]
    df_cali = pd.DataFrame(columns=['pred', 'actual', 'errorbar'])
    for i in range(8):
        temp = cohort[(cohort.propensity_score>=bins[i]) & (cohort.propensity_score<=bins[i+1])]
        x = np.mean(temp.propensity_score)
        y = len(temp[temp['treatment']==1])/len(temp)
        std = np.std(temp.propensity_score)
        df_cali.loc[len(df_cali)] = [x,y,1.96*std]

    fig = plt.figure()
    # plt.plot(df_cali.pred, df_cali.actual, '-o')
    plt.errorbar(df_cali.pred, df_cali.actual, yerr=df_cali.errorbar, fmt='o', ls='--')
    plt.plot([0, 1], [0, 1], ls='--')
    plt.xlabel('Mean predicted probability')
    plt.ylabel('Fraction of positives')
    plt.title('Calibration Plot for '+treatment)
    if name:
        plt.savefig(name)
    if not show:
        plt.close(fig)
    return fig

import pathlib
def generate_plots(model_config=None,model=None, outcome=None,build=False, show_results=False,log_name="logs"):
    if outcome == None:
        print(RaiseExceptionError("Outcome variable not specified"))
    feat_name = model_config["name"]
    #directory: treatment/outcome variable/feature/
    dir_name = f"Logs_2023/Grid search/{log_name}_{treatment.lower()}/{model_config['outcome']}/{model_config['feature']}/"
    pathlib.Path(dir_name).mkdir(parents=True, exist_ok=True) 
    
    f_name =  dir_name + feat_name


    
    if build and model_config==None:
        cohort, clf, params, score, metrics = build_model(featureset,outcome,model)
    else:
        model = model_config["model_num"]
        cohort = model_config["cohort"]
        clf = model_config["clf"]
        params = model_config["params"]
        score = model_config["score"]
        metrics = model_config["metrics"]
        accuracy, roc_auc, fpr, tpr = metrics
        roc_auc_metrics = (roc_auc, fpr, tpr)
        featureset = model_config["feature"]
        params = model_config["params"]
    model_index = model-1
    if model == 1:
        importances = clf.coef_[0]
    else:
        importances = clf.feature_importances_
    
    imp_name =  feat_imp[model_index]
    df_coef = pd.DataFrame(list(zip(feat_dict[outcome][featureset][1], importances)), columns=['feature',imp_name])
    feat_pd = df_coef.iloc[(-df_coef[imp_name].abs()).argsort()].head(100)
    feat_pd.to_csv(f_name + "_top_features.csv")

    if show_results:
        show=True
    else:
        show=False
        
    acc_pd = pd.DataFrame(np.array([[accuracy],[roc_auc]]).reshape(1,2),columns=["accuracy","roc_auc"])
    acc_pd.to_csv(f_name + "_metrics.csv")

    auc_name = f_name + "_roc_auc.png"
    compute_roc_auc(roc_auc_metrics,name=auc_name,show=show)
    
    r_name = f_name + "_reweighting.png"
    val, age_diff = reweighting(cohort,name=r_name,show=show)
    age_diff_pd = pd.DataFrame([age_diff], columns=["standardized age difference"])
    
    age_diff_pd.to_csv(f_name + "_age_diff.csv")
    
    p_name = f_name + "_propensity.png"
    p_plot = propensity(cohort,name=p_name,show=show)
    
    c_name = f_name + "_calibration.png"
    c_plot = calibration(cohort,name=c_name,show=show)
    
    
    param_df = pd.DataFrame(np.array(list(params.values())).reshape(1,len(params.values())),columns=list(params.keys()))
    param_df.to_csv(f_name + "_best_params.csv")
    
    if show_results:
        print(featureset.upper(), f"BEST MODEL ({model_list[model_index].upper()})")
        
        print(f"Accuracy: {accuracy}")
        print(f"ROC_AUC: {roc_auc:.4}")
        print(f"Best params: {params}")
        print(f"feat_size {df_coef.shape}")
        print("feature_importances")
        display(feat_pd.head(30))
        print("age reweighted standardized difference: ",age_diff)
    


##  Hyperparameter Tuning Models (optional)

In [ ]:
#sanity check we're getting both features
grid_params ={'censor' : {'outcome_names':['15','30','90'],'feats':feat_dict['censor']},
              'treatment' : {'outcome_names':[treatment_name],'feats': feat_dict['treatment']}}


#NOAH ADDED NEW GRID SEARCH FUNCTIONALITY FOR OUTCOME
outcomes = ['censor','treatment']
for outcome in outcomes:

    grid_features= list(grid_params[outcome]['feats'].keys())

In [ ]:
%%time
#choose the parameter model combo

#change grid_params if grid searching over treatment outcome instead of censor outcome

all_best_hyperparams = {}
grid_params ={'censor' : {'outcome_names':['15','30','90'],'feats':feat_dict['censor']},
              'treatment' : {'outcome_names':[treatment_name],'feats': feat_dict['treatment']}}


#NOAH ADDED NEW GRID SEARCH FUNCTIONALITY FOR OUTCOME
outcomes = ['censor','treatment']
for outcome in outcomes:

    grid_features= list(grid_params[outcome]['feats'].keys())
    grid_outcomes = [(k,v) for k,v in outcome_dict[outcome].items() if k in grid_params[outcome]['outcome_names']]


    feat_imp = ["Coefficient","Gini importance","Number of splits"]
    model_list = ["Logistic_Regression","Random_Forest","LGBM"]

    best_models = []
    all_models = []
    for out_name, out_val in grid_outcomes:
        y = out_val
        for f in grid_features:
            best_score = -float("inf")
            for m in range(1,4):
                #add outcome_name to model feature dictionary
                cohort, clf, params, score, metric = build_model(f,outcome,y,m,cv=True,folds=3)
                model_feature = {"name":f"{model_list[m-1]}","outcome":out_name, "score":score,"feature":f,"model_num":m,"metrics":metric,"params":params,"clf":clf,"cohort":cohort}
                all_models.append(model_feature)
                if score > best_score:
                    best_score = score
                    best_model_num = m
                    best_params = params
                    best_metric = metric
                    best_clf = clf
                    best_cohort = cohort

            print(f"{f} with {out_name} {outcome} outcome models DONE.")
            print("BEST MODEL WAS ", f"{model_list[best_model_num-1]} with ROC_AUC score: {best_score}")
            print(f"Best params are: {best_params}")
            best_model_feature = {"name":f"{model_list[best_model_num-1]}","score":best_score,"feature":f,"model_num":best_model_num,"metrics":best_metric,"params":best_params,"clf":best_clf,"cohort":best_cohort}
            best_models.append(best_model_feature)
            
            #loading the best params into dictionary
            best_params['clf'] = model_list[best_model_num-1]
            all_best_hyperparams.setdefault(f, {}).setdefault(treatment_name, {})[out_name] = best_params



In [ ]:
print(all_best_hyperparams)

In [ ]:
import pickle
pickle.dump(all_best_hyperparams,open(f'best_{treatment_name}_hyperparameters.pkl','wb'))

In [ ]:
## first we convert the clfs

## then we separate the dictionaries into the variables of interest using a dict comprehension
# first these: best_alternatives_cohort, best_second_line_cohort, best_second_line_omop, best_alternatives_omop
# then create these: best_D_params_cohort, best_A_params_cohort, best_D_params_omop, best_A_params_omop


In [ ]:
new_log_name = f"logs_{in_name}"
for outcome in outcomes:
    for model_feature_config in all_models:
        generate_plots(model_config=model_feature_config,outcome=outcome,show_results=False,log_name=new_log_name)

## ATE Code

## Run analysis

In [ ]:
#ATE helper functions
from joblib import Parallel, delayed

# define function that computes the IPTW estimator
def run_ps(df, y, features=None, params=None):
#     estimate the propensity score

    X = scipy.sparse.csr_matrix(df[features])
    T = df['treatment']
    X = MaxAbsScaler().fit_transform(X) #change to maxabs scaling
    ps = LogisticRegression(C=0.1, penalty='l1', solver='liblinear',tol=1e-4, max_iter=75).fit(X, T).predict_proba(X)[:, 1]

    weight = (T-ps) / (ps*(1-ps)) # define the weights
    y_ = df[y]
    z = df['treatment']
    
    df["ips"] = np.where(df['treatment'] == 1, 1 / ps, 1 / (1 - ps))
    df['ipsw'] = df[y]*df['ips']
    
    ey1 = z*y_/ps / sum(z/ps)
    ey0 = (1-z)*y_/(1-ps) / sum((1-z)/(1-ps))
    ate = ey1.sum()-ey0.sum()
    return [ate, ey0.sum(), ey1.sum()] # compute the ATE


def bootstrap(test, bootstrap_sample = 1, other_word = '', features=None, params=None):

    out = Parallel(n_jobs=15)(delayed(run_ps)(test.sample(frac=1, replace=True), y, features=features, params=params)
                              for _ in range(bootstrap_sample))

    #ates = np.array(ates)
    ates = np.array(out)[:,0]
    y0s = np.array(out)[:,1]
    y1s = np.array(out)[:,2]
    
    EY1 = np.round(y1s.mean(),4); EY0 = np.round(y0s.mean(),4)
    ATE = np.round(ates.mean(),4)
    CI95 = (np.round(np.percentile(ates, 2.5),4), np.round(np.percentile(ates, 97.5),4))
    
    # print "Showing results for", y, other_word
    print("ATE for", other_word, ":", ATE)
    print("95% C.I.: (", CI95[0], ", ", CI95[1], ")")
    print("EY1:", EY1)
    print("EY0:", EY0)
    return [EY1, EY0, ATE, CI95]

def run_ps2(data, params, verbose=True):
    X, T, y_ = data
    if verbose == True:
        print(f"Feature shape {X.shape}, Treatment shape {T.shape}, Outcome shape {y_.shape}")
        
    X = MaxAbsScaler().fit_transform(X) #change to maxabs scaling
    ps = lgb.LGBMClassifier().set_params(**params).fit(X, T).predict_proba(X)[:, 1]
    
    weight = (T-ps) / (ps*(1-ps)) # define the weights
    ey1 = T*y_/ps / sum(T/ps)
    ey0 = (1-T)*y_/(1-ps) / sum((1-T)/(1-ps))
    ate = ey1.sum()-ey0.sum()
    return [ate, ey0.sum(), ey1.sum()] # compute the ATE

    
def sparse_bootstrap(X_sparse, T, y, bootstrap_sample = 1, other_word = '', params=None):
    out = Parallel(n_jobs=15)(delayed(run_ps2)(resample(X_sparse, T, filtered_cohort[y]), params=params)
                              for _ in range(bootstrap_sample))
    #ates = np.array(ates)
    ates = np.array(out)[:,0]
    y0s = np.array(out)[:,1]
    y1s = np.array(out)[:,2]
    
    EY1 = np.round(y1s.mean(),4); EY0 = np.round(y0s.mean(),4)
    ATE = np.round(ates.mean(),4)
    CI95 = (np.round(np.percentile(ates, 2.5),4), np.round(np.percentile(ates, 97.5),4))
    
    # print "Showing results for", y, other_word
    print("ATE for", other_word, ":", ATE)
    print("95% C.I.: (", CI95[0], ", ", CI95[1], ")")
    print("EY1:", EY1)
    print("EY0:", EY0)
    return [EY1, EY0, ATE, CI95]

def censor_bootstrap(X_sparse, T, y, D, bootstrap_sample = 1, other_word = '', params:tuple=None):
    
    out = Parallel(n_jobs=15)(delayed(run_ps3)(resample(X_sparse, T, filtered_cohort[y], D), params=params)
                              for _ in range(bootstrap_sample))
    #ates = np.array(ates)
    ates = np.array(out)[:,0]
    y0s = np.array(out)[:,1]
    y1s = np.array(out)[:,2]
    
    EY1 = np.round(y1s.mean(),6); EY0 = np.round(y0s.mean(),6)
    ATE = np.round(ates.mean(),6)
    CI95 = (np.round(np.percentile(ates, 2.5),6), np.round(np.percentile(ates, 97.5),6))
    
    # print "Showing results for", y, other_word
    print("ATE for", other_word, ":", ATE)
    print("95% C.I.: (", CI95[0], ", ", CI95[1], ")")
    print("EY1:", EY1)
    print("EY0:", EY0)
    return [EY1, EY0, ATE, CI95]

def run_ps3(data, params, verbose=False, prop = False):
    #computes the IPTW estimator taking into account censoring
    params_copy = copy.deepcopy(params)
    A_params, D_params = params_copy
    clf_A = A_params.pop('clf', None)
    clf_D = D_params.pop('clf', None)

    #check to make sure a classifier was provided
    assert not isinstance(clf_A,type(None)), "no treatment classifier provided"
    assert not isinstance(clf_D,type(None)), "no censor classifier provided"
    

    # D = 1 if followed for at least 30 days and 0 if less than 30 days
    # A = 1 if given second line and 0 if given first line
    # X are the covariates
    # Y is the outcome (which is treated as missing when D = 0)
    # (1) fit propensity model P(A=1| X)
    # (2) fit model for non-missing P(D=1 | X, A)
    # The estimated ATE for this bootstrap sample is then, averaging over all patients with D=1
    # E[ A_i Y_i / [P(A=1 | X_i) P(D=1 | A, X_i)] - (1-A_i) Y_i / [P(A=0 | X_i) P(D=1 | A, X_i)] ]

    X, A, Y, D = data
    if verbose == True:
        print(f"Feature shape {X.shape}, Treatment shape {A.shape}, Outcome shape {Y.shape}, D shape {D.shape}")
    

    #just the X variable
    X_1 = MaxAbsScaler().fit_transform(X) #change to maxabs scaling
    #the X and A variables

    X_2 = vstack([X_1.T, scipy.sparse.csr_matrix(A)]).T
    
    
    

    #fit propensity model
    ps = clf_A.set_params(**A_params).fit(X_1, A).predict_proba(scipy.sparse.csr_matrix(X_1))[:, 1]
    if prop:
        return(ps)
    else:
        #fit model for non-missing D
        ps_d = clf_D.set_params(**D_params).fit(X_2, D).predict_proba(scipy.sparse.csr_matrix(X_2))[:, 1]

        #filter for patients with D=1
        ps_d_filtered = ps_d[D==1]
        ps_filtered = ps[D==1]
        Y_filtered = Y[D==1]
        A_filtered = A[D==1]
        
        
        #compute e1 and e0
        ey1 = A_filtered*Y_filtered/(ps_filtered * ps_d_filtered) / sum(A_filtered/(ps_filtered * ps_d_filtered))
        ey0 = (1-A_filtered)*Y_filtered/((1-ps_filtered) * ps_d_filtered) / sum((1-A_filtered)/((1-ps_filtered) * ps_d_filtered))

        ate = ey1.sum()-ey0.sum()
        return [ate, ey0.sum(), ey1.sum()] # compute the ATE





## Define outcomes

In [ ]:
adverse_event_outcomes = ['AE_c_diff',
 'AE_skin', 'AE_GI', 'AE_AKI',
 'AE_other']

negative_control_outcomes = ['fibro_' + m + '_mon_outcome' for m in ['1','3','6']] + \
 ['hernia_' + m + '_mon_outcome' for m in ['1','3','6']] + \
 ['fracture_' + m + '_mon_outcome' for m in ['1','3','6']]

#removed the below code in new iteration given that feature was added
# filtered_cohort['AE_any'] = filtered_cohort[adverse_event_outcomes].any(axis='columns').astype(int)
adverse_event_outcomes.append('AE_any')

efficacy_outcomes = [f for f in filtered_cohort.columns if f.startswith('t_') and f.endswith('_bin')]
efficacy_outcomes = [f for f in efficacy_outcomes if f!='t_condition_occurence_ids']

print(adverse_event_outcomes)
print(negative_control_outcomes)
print(efficacy_outcomes)

out_time_dict = {'AE_c_diff' : '90', 'AE_skin' : '30', 'AE_other' : '30', 'AE_GI' : '15', 'AE_AKI' : '30','AE_any':'90'}
out_time_dict.update({k: '30' for k in negative_control_outcomes if '1' in k})
out_time_dict.update({k: '90' for k in negative_control_outcomes if '3' in k})
out_time_dict.update({k: '90' for k in negative_control_outcomes if '6' in k})
out_time_dict.update({k: '30' for k in ['t_bin', 't_uti_bin', 't_neph_bin', 't_sepsis_bin', 't_i_uti_bin', 't_i_neph_bin', 't_i_sepsis_bin', 't_i_bin']})


from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

sklearn_model_map = {'Logistic_Regression' : LogisticRegression(),
     'Random_Forest' : RandomForestClassifier(),
     'LGBM' : lgb.LGBMClassifier()}
features = ["cohort_features","omop_1_year_features"]

def update_nested_dict(d, target_key, mapping_dict=sklearn_model_map):
    for key, value in d.copy().items():
        if key == 'penalty':
            if d[key] == 'l1':
                d['solver'] = 'liblinear'
        if key == target_key:
            if d[key] != 'Random_Forest':
                d['verbose'] = -1
            d[key] = mapping_dict[d[key]]
        elif isinstance(value, dict):
            d[key] = update_nested_dict(value, target_key)
    return d

# def add_penalty_params()

def format_parameter_dictionaries(p_alt,p_sec):
    treatments = ['alternatives','second_line']
    
    
    ## create A params
    best_A_params = dict()
    best_A_params['alternatives'] = p_alt['alternatives']['alternatives']
    best_A_params['second_line'] = p_sec['second_line']['second_line']

    
    #create D params
    best_D_params = dict()
    for p_group,t_group in zip([p_alt,p_sec],treatments):
        best_D_params[t_group] = {treat:params for treat,params in p_group[t_group].items() if treat!= t_group}

    
    #convert the clf keys into correct values
    best_A_params = update_nested_dict(best_A_params,'clf')
    best_D_params = update_nested_dict(best_D_params,'clf')
    return best_A_params,best_D_params



alternatives = pd.read_pickle("../../Data/best_alternatives_hyperparameters.pkl")
second_line = pd.read_pickle("../../Data/best_second_line_hyperparameters.pkl")

best_A_params_cohort,best_D_params_cohort = format_parameter_dictionaries(alternatives['cohort_features'],second_line['cohort_features'])

best_A_params_omop,best_D_params_omop = format_parameter_dictionaries(alternatives['omop_1_year_features'],second_line['omop_1_year_features'])

ATE_A = {"cohort_features":best_A_params_cohort, "omop_1_year_features":best_A_params_omop}
ATE_D = {"cohort_features":best_D_params_cohort, "omop_1_year_features":best_D_params_omop}



## 03042024 results (using bootstrap=1000)

In [ ]:
filtered_cohort.antibiotic_type.value_counts().sum() - 21140

In [ ]:
filtered_cohort.antibiotic_type.value_counts(normalize=True)

In [ ]:
%%time

from collections import defaultdict
discard = True
results_df = defaultdict(dict)

A = outcome_dict['treatment'][treatment_name]

for feature in features:
    print(feature)
    results = pd.DataFrame(columns=['Outcome', 'E[Y_1]', 'E[Y_0]', 'ATE', '95% C.I.'])
    for y in efficacy_outcomes:
        X = feat_dict['treatment'][feature][0]
        time = out_time_dict[y]
        D = outcome_dict['censor'][time]
        params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
        if discard:
            ps = run_ps3((X, A, filtered_cohort[y], D), params, prop = True)
            print(len(ps))
            kp = np.where((ps >= 0.05) * (ps <= 0.95))
            print(len(kp[0]))
            temp_filtered_cohort = filtered_cohort.copy()
            filtered_cohort = filtered_cohort.iloc[kp]
            params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
            summary = censor_bootstrap(X[kp], A.iloc[kp], y, D.iloc[kp], bootstrap_sample=1000, other_word=y, params=params)
            filtered_cohort = temp_filtered_cohort.copy()
        else:
            summary = censor_bootstrap(X, A, y, D, bootstrap_sample=1000, other_word=y, params=params)
        results.loc[len(results)] = [y] + summary
    
    results_df['t_efficacy'][feature] = results
    print("\n\n")
    
for feature in features:
    print(feature)
    results = pd.DataFrame(columns=['Outcome', 'E[Y_1]', 'E[Y_0]', 'ATE', '95% C.I.'])
    for y in adverse_event_outcomes:
        X = feat_dict['treatment'][feature][0]
        time = out_time_dict[y]
        D = outcome_dict['censor'][time]
        params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
        if discard:
            ps = run_ps3((X, A, filtered_cohort[y], D), params, prop = True)

            kp = np.where((ps >= 0.05) * (ps <= 0.95))
            temp_filtered_cohort = filtered_cohort.copy()
            filtered_cohort = filtered_cohort.iloc[kp]
            params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
            summary = censor_bootstrap(X[kp], A.iloc[kp], y, D.iloc[kp], bootstrap_sample=1000, other_word=y, params=params)
            filtered_cohort = temp_filtered_cohort.copy()
        else:
            summary = censor_bootstrap(X, A, y, D, bootstrap_sample=1000, other_word=y, params=params)
        results.loc[len(results)] = [y] + summary
    
    results_df['adverse_events'][feature] = results
    print("\n\n")
    

for feature in features:
    print(feature)
    results = pd.DataFrame(columns=['Outcome', 'E[Y_1]', 'E[Y_0]', 'ATE', '95% C.I.'])
    for y in negative_control_outcomes:
        X = feat_dict['treatment'][feature][0]
        time = out_time_dict[y]
        D = outcome_dict['censor'][time]
        params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
        if discard:
            ps = run_ps3((X, A, filtered_cohort[y], D), params, prop = True)
            kp = np.where((ps >= 0.05) * (ps <= 0.95))
            temp_filtered_cohort = filtered_cohort.copy()
            filtered_cohort = filtered_cohort.iloc[kp]
            params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
            summary = censor_bootstrap(X[kp], A.iloc[kp], y, D.iloc[kp], bootstrap_sample=1000, other_word=y, params=params)
            filtered_cohort = temp_filtered_cohort.copy()
        else:
            summary = censor_bootstrap(X, A, y, D, bootstrap_sample=1000, other_word=y, params=params)
        results.loc[len(results)] = [y] + summary
    
    results_df['negative_controls'][feature] = results
    print("\n\n")

In [ ]:
ps.shape

In [ ]:
kp[0]

In [ ]:
%%time

from collections import defaultdict
discard = True
results_df = defaultdict(dict)

A = outcome_dict['treatment'][treatment_name]

for feature in features:
    print(feature)
    results = pd.DataFrame(columns=['Outcome', 'E[Y_1]', 'E[Y_0]', 'ATE', '95% C.I.'])
    for y in efficacy_outcomes:
        X = feat_dict['treatment'][feature][0]
        time = out_time_dict[y]
        D = outcome_dict['censor'][time]
        params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
        if discard:
            ps = run_ps3((X, A, filtered_cohort[y], D), params, prop = True)
            kp = np.where((ps >= 0.05) * (ps <= 0.95))
            temp_filtered_cohort = filtered_cohort.copy()
            filtered_cohort = filtered_cohort.iloc[kp]
            params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
            summary = censor_bootstrap(X[kp], A.iloc[kp], y, D.iloc[kp], bootstrap_sample=1000, other_word=y, params=params)
            filtered_cohort = temp_filtered_cohort.copy()
        else:
            summary = censor_bootstrap(X, A, y, D, bootstrap_sample=1000, other_word=y, params=params)
        results.loc[len(results)] = [y] + summary
    
    results_df['t_efficacy'][feature] = results
    print("\n\n")
    
for feature in features:
    print(feature)
    results = pd.DataFrame(columns=['Outcome', 'E[Y_1]', 'E[Y_0]', 'ATE', '95% C.I.'])
    for y in adverse_event_outcomes:
        X = feat_dict['treatment'][feature][0]
        time = out_time_dict[y]
        D = outcome_dict['censor'][time]
        params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
        if discard:
            ps = run_ps3((X, A, filtered_cohort[y], D), params, prop = True)
            kp = np.where((ps >= 0.05) * (ps <= 0.95))
            temp_filtered_cohort = filtered_cohort.copy()
            filtered_cohort = filtered_cohort.iloc[kp]
            params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
            summary = censor_bootstrap(X[kp], A.iloc[kp], y, D.iloc[kp], bootstrap_sample=1000, other_word=y, params=params)
            filtered_cohort = temp_filtered_cohort.copy()
        else:
            summary = censor_bootstrap(X, A, y, D, bootstrap_sample=1000, other_word=y, params=params)
        results.loc[len(results)] = [y] + summary
    
    results_df['adverse_events'][feature] = results
    print("\n\n")
    

for feature in features:
    print(feature)
    results = pd.DataFrame(columns=['Outcome', 'E[Y_1]', 'E[Y_0]', 'ATE', '95% C.I.'])
    for y in negative_control_outcomes:
        X = feat_dict['treatment'][feature][0]
        time = out_time_dict[y]
        D = outcome_dict['censor'][time]
        params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
        if discard:
            ps = run_ps3((X, A, filtered_cohort[y], D), params, prop = True)
            kp = np.where((ps >= 0.05) * (ps <= 0.95))
            temp_filtered_cohort = filtered_cohort.copy()
            filtered_cohort = filtered_cohort.iloc[kp]
            params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
            summary = censor_bootstrap(X[kp], A.iloc[kp], y, D.iloc[kp], bootstrap_sample=1000, other_word=y, params=params)
            filtered_cohort = temp_filtered_cohort.copy()
        else:
            summary = censor_bootstrap(X, A, y, D, bootstrap_sample=1000, other_word=y, params=params)
        results.loc[len(results)] = [y] + summary
    
    results_df['negative_controls'][feature] = results
    print("\n\n")

In [ ]:
%%time

from collections import defaultdict
discard = True
results_df = defaultdict(dict)

A = outcome_dict['treatment'][treatment_name]

for feature in features:
    print(feature)
    results = pd.DataFrame(columns=['Outcome', 'E[Y_1]', 'E[Y_0]', 'ATE', '95% C.I.'])
    for y in efficacy_outcomes:
        X = feat_dict['treatment'][feature][0]
        time = out_time_dict[y]
        D = outcome_dict['censor'][time]
        params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
        if discard:
            ps = run_ps3((X, A, filtered_cohort[y], D), params, prop = True)
            kp = np.where((ps >= 0.05) * (ps <= 0.95))
            temp_filtered_cohort = filtered_cohort.copy()
            filtered_cohort = filtered_cohort.iloc[kp]
            params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
            summary = censor_bootstrap(X[kp], A.iloc[kp], y, D.iloc[kp], bootstrap_sample=100, other_word=y, params=params)
            filtered_cohort = temp_filtered_cohort.copy()
        else:
            summary = censor_bootstrap(X, A, y, D, bootstrap_sample=100, other_word=y, params=params)
        results.loc[len(results)] = [y] + summary
    
    results_df['t_efficacy'][feature] = results
    print("\n\n")
    
for feature in features:
    print(feature)
    results = pd.DataFrame(columns=['Outcome', 'E[Y_1]', 'E[Y_0]', 'ATE', '95% C.I.'])
    for y in adverse_event_outcomes:
        X = feat_dict['treatment'][feature][0]
        time = out_time_dict[y]
        D = outcome_dict['censor'][time]
        params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
        if discard:
            ps = run_ps3((X, A, filtered_cohort[y], D), params, prop = True)
            kp = np.where((ps >= 0.05) * (ps <= 0.95))
            temp_filtered_cohort = filtered_cohort.copy()
            filtered_cohort = filtered_cohort.iloc[kp]
            params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
            summary = censor_bootstrap(X[kp], A.iloc[kp], y, D.iloc[kp], bootstrap_sample=100, other_word=y, params=params)
            filtered_cohort = temp_filtered_cohort.copy()
        else:
            summary = censor_bootstrap(X, A, y, D, bootstrap_sample=100, other_word=y, params=params)
        results.loc[len(results)] = [y] + summary
    
    results_df['adverse_events'][feature] = results
    print("\n\n")
    

for feature in features:
    print(feature)
    results = pd.DataFrame(columns=['Outcome', 'E[Y_1]', 'E[Y_0]', 'ATE', '95% C.I.'])
    for y in negative_control_outcomes:
        X = feat_dict['treatment'][feature][0]
        time = out_time_dict[y]
        D = outcome_dict['censor'][time]
        params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
        if discard:
            ps = run_ps3((X, A, filtered_cohort[y], D), params, prop = True)
            kp = np.where((ps >= 0.05) * (ps <= 0.95))
            temp_filtered_cohort = filtered_cohort.copy()
            filtered_cohort = filtered_cohort.iloc[kp]
            params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
            summary = censor_bootstrap(X[kp], A.iloc[kp], y, D.iloc[kp], bootstrap_sample=100, other_word=y, params=params)
            filtered_cohort = temp_filtered_cohort.copy()
        else:
            summary = censor_bootstrap(X, A, y, D, bootstrap_sample=100, other_word=y, params=params)
        results.loc[len(results)] = [y] + summary
    
    results_df['negative_controls'][feature] = results
    print("\n\n")

In [ ]:
import pickle
pickle.dump(results_df,open(f'final_{treatment_name}_03042024_results.pkl','wb'))

### V4 Config

In [ ]:
#Setup config from grid search
#cohort
best_A_params_cohort = {'alternatives':{'random_state': 0, 'boosting_type': 'gbdt', 'learning_rate': 0.1, 'num_leaves': 10, 'verbose': -1, "clf":lgb.LGBMClassifier()},'second_line':{'random_state': 0, 'boosting_type': 'gbdt', 'learning_rate': 0.05, 'num_leaves': 50, 'verbose': -1, "clf":lgb.LGBMClassifier()}} # best params for second line change for alternatives
best_D_params_cohort = {'alternatives':{'15': {'max_features': 'auto', 'min_samples_leaf': 75, 'n_estimators': 140, "clf":RandomForestClassifier()},
                                 '30':{'C': 0.25, 'penalty': 'l1',"solver":'liblinear', 'verbose': -1, "clf":LogisticRegression()},
                                 '90':{'boosting_type': 'gbdt', 'learning_rate': 0.05, 'num_leaves': 10, 'verbose': -1, "clf":lgb.LGBMClassifier()}},
                'second_line':{'15':{'boosting_type': 'gbdt', 'learning_rate': 0.01, 'num_leaves': 10, 'verbose': -1, "clf":lgb.LGBMClassifier()},
                               '30': {'boosting_type': 'gbdt', 'learning_rate': 0.05, 'num_leaves': 10, 'verbose': -1, "clf":lgb.LGBMClassifier()},
                               '90':{'boosting_type': 'gbdt', 'learning_rate': 0.05, 'num_leaves': 10, 'verbose': -1, "clf":lgb.LGBMClassifier()}}
                }
features = ["cohort_features","omop_1_year_features"]

#omop

best_A_params_omop = {'alternatives':{'random_state': 0, 'boosting_type': 'gbdt', 'learning_rate': 0.05, 'num_leaves': 50, 'verbose': -1, "clf":lgb.LGBMClassifier()},'second_line':{'random_state': 0, 'boosting_type': 'dart', 'learning_rate': 0.1, 'num_leaves': 50, 'verbose': -1, "clf":lgb.LGBMClassifier()}} # best params for second line change for alternatives
best_D_params_omop = {'alternatives':{'15':{'max_features': 'auto', 'min_samples_leaf': 50, 'n_estimators': 100, "clf":RandomForestClassifier()},
                                 '30':{'boosting_type': 'gbdt', 'learning_rate': 0.05, 'num_leaves': 250, 'verbose': -1, "clf":lgb.LGBMClassifier()},
                                 '90':{'max_features': 'auto', 'min_samples_leaf': 75, 'n_estimators': 140, "clf":RandomForestClassifier()}},
                'second_line':{'15':{'max_features': 'auto', 'min_samples_leaf': 75, 'n_estimators': 120, "clf":RandomForestClassifier()},
                               '30':{'max_features': 'auto', 'min_samples_leaf': 75, 'n_estimators': 100, "clf":RandomForestClassifier()},
                               '90':{'max_features': 'auto', 'min_samples_leaf': 75, 'n_estimators': 120, "clf":RandomForestClassifier()}}
                }
                      
ATE_A = {"cohort_features":best_A_params_cohort, "omop_1_year_features":best_A_params_omop}
ATE_D = {"cohort_features":best_D_params_cohort, "omop_1_year_features":best_D_params_omop}

### V2 config

In [ ]:
#Setup config from grid search
#cohort
best_A_params_cohort = {'alternatives':{'random_state': 0, 'boosting_type': 'gbdt', 'learning_rate': 0.1, 'num_leaves': 10, 'verbose': -1, "clf":lgb.LGBMClassifier()},'second_line':{'random_state': 0, 'boosting_type': 'gbdt', 'learning_rate': 0.1, 'num_leaves': 10, 'verbose': -1, "clf":lgb.LGBMClassifier()}} # best params for second line change for alternatives
best_D_params_cohort = {'alternatives':{'15': {'max_features': 'auto', 'min_samples_leaf': 50, 'n_estimators': 140, "clf":RandomForestClassifier()},
                                 '30':{'max_features': 'auto', 'min_samples_leaf': 50, 'n_estimators': 60, "clf":RandomForestClassifier()},
                                 '90':{'boosting_type': 'gbdt', 'learning_rate': 0.05, 'num_leaves': 10, 'verbose': -1, "clf":lgb.LGBMClassifier()}},
                'second_line':{'15':{'C': 0.1, 'penalty': 'l1',"solver":'liblinear', 'verbose': -1, "clf":LogisticRegression()},
                               '30': {'boosting_type': 'dart', 'learning_rate': 0.1, 'num_leaves': 10, 'verbose': -1, "clf":lgb.LGBMClassifier()},
                               '90':{'boosting_type': 'gbdt', 'learning_rate': 0.05, 'num_leaves': 10, 'verbose': -1, "clf":lgb.LGBMClassifier()}}
                }
features = ["cohort_features","omop_1_year_features"]

#omop

best_A_params_omop = {'alternatives':{'random_state': 0, 'boosting_type': 'gbdt', 'learning_rate': 0.05, 'num_leaves': 50, 'verbose': -1, "clf":lgb.LGBMClassifier()},'second_line':{'random_state': 0, 'boosting_type': 'dart', 'learning_rate': 0.1, 'num_leaves': 50, 'verbose': -1, "clf":lgb.LGBMClassifier()}} # best params for second line change for alternatives
best_D_params_omop = {'alternatives':{'15':{'max_features': 'auto', 'min_samples_leaf': 50, 'n_estimators': 100, "clf":RandomForestClassifier()},
                                 '30':{'boosting_type': 'gbdt', 'learning_rate': 0.05, 'num_leaves': 250, 'verbose': -1, "clf":lgb.LGBMClassifier()},
                                 '90':{'max_features': 'auto', 'min_samples_leaf': 75, 'n_estimators': 140, "clf":RandomForestClassifier()}},
                'second_line':{'15':{'max_features': 'auto', 'min_samples_leaf': 75, 'n_estimators': 120, "clf":RandomForestClassifier()},
                               '30':{'max_features': 'auto', 'min_samples_leaf': 75, 'n_estimators': 100, "clf":RandomForestClassifier()},
                               '90':{'max_features': 'auto', 'min_samples_leaf': 75, 'n_estimators': 120, "clf":RandomForestClassifier()}}
                }
                      
ATE_A = {"cohort_features":best_A_params_cohort, "omop_1_year_features":best_A_params_omop}
ATE_D = {"cohort_features":best_D_params_cohort, "omop_1_year_features":best_D_params_omop}

### NOAH REPLICATION OF MING-CHIEH RESULTS FOR SECOND-LINE

### Have to remove the first feature here

In [ ]:
# #removing cohort features.  12-14 NOAH COMMENTED THIS OUT BECAUSE I THINK FOR OMOP IT IS PROBABLY NEEDED
# features.pop(1)
# print(features)

### V4 (start)

In [ ]:
%%time

discard = True

A = outcome_dict['treatment'][treatment_name]

for feature in features:
    print(feature)
    results = pd.DataFrame(columns=['Outcome', 'E[Y_1]', 'E[Y_0]', 'ATE', '95% C.I.'])
    for y in efficacy_outcomes:
        X = feat_dict['treatment'][feature][0]
        time = out_time_dict[y]
        D = outcome_dict['censor'][time]
        params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
        if discard:
            ps = run_ps3((X, A, filtered_cohort[y], D), params, prop = True)
            kp = np.where((ps >= 0.05) * (ps <= 0.95))
            temp_filtered_cohort = filtered_cohort.copy()
            filtered_cohort = filtered_cohort.iloc[kp]
            params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
            summary = censor_bootstrap(X[kp], A.iloc[kp], y, D.iloc[kp], bootstrap_sample=100, other_word=y, params=params)
            filtered_cohort = temp_filtered_cohort.copy()
        else:
            summary = censor_bootstrap(X, A, y, D, bootstrap_sample=100, other_word=y, params=params)
        results.loc[len(results)] = [y] + summary
    
    results_df['t_efficacy'][feature] = results
    print("\n\n")

In [ ]:
discard = True

#run bootstrap

A = outcome_dict['treatment'][treatment_name]

results_df = defaultdict(dict)

for feature in features:
    print(feature)
    results = pd.DataFrame(columns=['Outcome', 'E[Y_1]', 'E[Y_0]', 'ATE', '95% C.I.'])
    for y in adverse_event_outcomes:
        X = feat_dict['treatment'][feature][0]
        time = out_time_dict[y]
        D = outcome_dict['censor'][time]
        params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
        if discard:
            ps = run_ps3((X, A, filtered_cohort[y], D), params, prop = True)
            kp = np.where((ps >= 0.05) * (ps <= 0.95))
            temp_filtered_cohort = filtered_cohort.copy()
            filtered_cohort = filtered_cohort.iloc[kp]
            params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
            summary = censor_bootstrap(X[kp], A.iloc[kp], y, D.iloc[kp], bootstrap_sample=100, other_word=y, params=params)
            filtered_cohort = temp_filtered_cohort.copy()
        else:
            summary = censor_bootstrap(X, A, y, D, bootstrap_sample=100, other_word=y, params=params)
        results.loc[len(results)] = [y] + summary
    
    results_df['adverse_events'][feature] = results
    print("\n\n")

### V4 (end)

### V2 (start)

In [ ]:
discard = True

#run bootstrap

A = outcome_dict['treatment'][treatment_name]

results_df = defaultdict(dict)

for feature in features:
    print(feature)
    results = pd.DataFrame(columns=['Outcome', 'E[Y_1]', 'E[Y_0]', 'ATE', '95% C.I.'])
    for y in adverse_event_outcomes:
        X = feat_dict['treatment'][feature][0]
        time = out_time_dict[y]
        D = outcome_dict['censor'][time]
        params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
        if discard:
            ps = run_ps3((X, A, filtered_cohort[y], D), params, prop = True)
            kp = np.where((ps >= 0.05) * (ps <= 0.95))
            temp_filtered_cohort = filtered_cohort.copy()
            filtered_cohort = filtered_cohort.iloc[kp]
            params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
            summary = censor_bootstrap(X[kp], A.iloc[kp], y, D.iloc[kp], bootstrap_sample=100, other_word=y, params=params)
            filtered_cohort = temp_filtered_cohort.copy()
        else:
            summary = censor_bootstrap(X, A, y, D, bootstrap_sample=100, other_word=y, params=params)
        results.loc[len(results)] = [y] + summary
    
    results_df['adverse_events'][feature] = results
    print("\n\n")

In [ ]:
%%time

discard = True

A = outcome_dict['treatment'][treatment_name]

for feature in features:
    print(feature)
    results = pd.DataFrame(columns=['Outcome', 'E[Y_1]', 'E[Y_0]', 'ATE', '95% C.I.'])
    for y in efficacy_outcomes:
        X = feat_dict['treatment'][feature][0]
        time = out_time_dict[y]
        D = outcome_dict['censor'][time]
        params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
        if discard:
            ps = run_ps3((X, A, filtered_cohort[y], D), params, prop = True)
            kp = np.where((ps >= 0.05) * (ps <= 0.95))
            temp_filtered_cohort = filtered_cohort.copy()
            filtered_cohort = filtered_cohort.iloc[kp]
            params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
            summary = censor_bootstrap(X[kp], A.iloc[kp], y, D.iloc[kp], bootstrap_sample=100, other_word=y, params=params)
            filtered_cohort = temp_filtered_cohort.copy()
        else:
            summary = censor_bootstrap(X, A, y, D, bootstrap_sample=100, other_word=y, params=params)
        results.loc[len(results)] = [y] + summary
    
    results_df['t_efficacy'][feature] = results
    print("\n\n")

### V2 (end)

### Have to reinitialize filtered_cohort to use the omp version

In [ ]:
## in order to get it to work, you need to make sure that filtered_cohort is the one from omop.  I think the notebook would benefit from changing that name.

In [ ]:
discard = True

#run bootstrap

A = outcome_dict['treatment'][treatment_name]

results_df = defaultdict(dict)

for feature in features:
    print(feature)
    results = pd.DataFrame(columns=['Outcome', 'E[Y_1]', 'E[Y_0]', 'ATE', '95% C.I.'])
    for y in adverse_event_outcomes:
        X = feat_dict['treatment'][feature][0]
        time = out_time_dict[y]
        D = outcome_dict['censor'][time]
        params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
        if discard:
            ps = run_ps3((X, A, filtered_cohort[y], D), params, prop = True)
            kp = np.where((ps >= 0.05) * (ps <= 0.95))
            temp_filtered_cohort = filtered_cohort.copy()
            filtered_cohort = filtered_cohort.iloc[kp]
            params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
            summary = censor_bootstrap(X[kp], A.iloc[kp], y, D.iloc[kp], bootstrap_sample=100, other_word=y, params=params)
            filtered_cohort = temp_filtered_cohort.copy()
        else:
            summary = censor_bootstrap(X, A, y, D, bootstrap_sample=100, other_word=y, params=params)
        results.loc[len(results)] = [y] + summary
    
    results_df['adverse_events'][feature] = results
    print("\n\n")

In [ ]:
%%time

discard = True

#run bootstrap

A = outcome_dict['treatment'][treatment_name]

for feature in features:
    print(feature)
    results = pd.DataFrame(columns=['Outcome', 'E[Y_1]', 'E[Y_0]', 'ATE', '95% C.I.'])
    for y in negative_control_outcomes:
        X = feat_dict['treatment'][feature][0]
        time = out_time_dict[y]
        D = outcome_dict['censor'][time]
        params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
        if discard:
            ps = run_ps3((X, A, filtered_cohort[y], D), params, prop = True)
            kp = np.where((ps >= 0.05) * (ps <= 0.95))
            temp_filtered_cohort = filtered_cohort.copy()
            filtered_cohort = filtered_cohort.iloc[kp]
            params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
            summary = censor_bootstrap(X[kp], A.iloc[kp], y, D.iloc[kp], bootstrap_sample=100, other_word=y, params=params)
            filtered_cohort = temp_filtered_cohort.copy()
        else:
            summary = censor_bootstrap(X, A, y, D, bootstrap_sample=100, other_word=y, params=params)
        results.loc[len(results)] = [y] + summary
    
    results_df['negative_controls'][feature] = results
    print("\n\n")

In [ ]:
%%time

discard = True

A = outcome_dict['treatment'][treatment_name]

for feature in features:
    print(feature)
    results = pd.DataFrame(columns=['Outcome', 'E[Y_1]', 'E[Y_0]', 'ATE', '95% C.I.'])
    for y in efficacy_outcomes:
        X = feat_dict['treatment'][feature][0]
        time = out_time_dict[y]
        D = outcome_dict['censor'][time]
        params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
        if discard:
            ps = run_ps3((X, A, filtered_cohort[y], D), params, prop = True)
            kp = np.where((ps >= 0.05) * (ps <= 0.95))
            temp_filtered_cohort = filtered_cohort.copy()
            filtered_cohort = filtered_cohort.iloc[kp]
            params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
            summary = censor_bootstrap(X[kp], A.iloc[kp], y, D.iloc[kp], bootstrap_sample=100, other_word=y, params=params)
            filtered_cohort = temp_filtered_cohort.copy()
        else:
            summary = censor_bootstrap(X, A, y, D, bootstrap_sample=100, other_word=y, params=params)
        results.loc[len(results)] = [y] + summary
    
    results_df['t_efficacy'][feature] = results
    print("\n\n")

### OLD MING-CHIEH RESULTS FOR ALTERNATIVES

In [ ]:
%%time

discard = True

#run bootstrap

A = outcome_dict['treatment'][treatment_name]

results_df = defaultdict(dict)

for feature in features:
    print(feature)
    results = pd.DataFrame(columns=['Outcome', 'E[Y_1]', 'E[Y_0]', 'ATE', '95% C.I.'])
    for y in adverse_event_outcomes:
        X = feat_dict['treatment'][feature][0]
        time = out_time_dict[y]
        D = outcome_dict['censor'][time]
        params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
        if discard:
            ps = run_ps3((X, A, filtered_cohort[y], D), params, prop = True)
            kp = np.where((ps >= 0.05) * (ps <= 0.95))
            temp_filtered_cohort = filtered_cohort.copy()
            filtered_cohort = filtered_cohort.iloc[kp]
            params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
            summary = censor_bootstrap(X[kp], A.iloc[kp], y, D.iloc[kp], bootstrap_sample=100, other_word=y, params=params)
            filtered_cohort = temp_filtered_cohort.copy()
        else:
            summary = censor_bootstrap(X, A, y, D, bootstrap_sample=100, other_word=y, params=params)
        results.loc[len(results)] = [y] + summary
    
    results_df['adverse_events'][feature] = results
    print("\n\n")

In [ ]:
%%time

discard = True

#run bootstrap

A = outcome_dict['treatment'][treatment_name]

for feature in features:
    print(feature)
    results = pd.DataFrame(columns=['Outcome', 'E[Y_1]', 'E[Y_0]', 'ATE', '95% C.I.'])
    for y in negative_control_outcomes:
        X = feat_dict['treatment'][feature][0]
        time = out_time_dict[y]
        D = outcome_dict['censor'][time]
        params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
        if discard:
            ps = run_ps3((X, A, filtered_cohort[y], D), params, prop = True)
            kp = np.where((ps >= 0.05) * (ps <= 0.95))
            temp_filtered_cohort = filtered_cohort.copy()
            filtered_cohort = filtered_cohort.iloc[kp]
            params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
            summary = censor_bootstrap(X[kp], A.iloc[kp], y, D.iloc[kp], bootstrap_sample=100, other_word=y, params=params)
            filtered_cohort = temp_filtered_cohort.copy()
        else:
            summary = censor_bootstrap(X, A, y, D, bootstrap_sample=100, other_word=y, params=params)
        results.loc[len(results)] = [y] + summary
    
    results_df['negative_controls'][feature] = results
    print("\n\n")

In [ ]:
%%time

discard = True

A = outcome_dict['treatment'][treatment_name]

for feature in features:
    print(feature)
    results = pd.DataFrame(columns=['Outcome', 'E[Y_1]', 'E[Y_0]', 'ATE', '95% C.I.'])
    for y in efficacy_outcomes:
        X = feat_dict['treatment'][feature][0]
        time = out_time_dict[y]
        D = outcome_dict['censor'][time]
        params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
        if discard:
            ps = run_ps3((X, A, filtered_cohort[y], D), params, prop = True)
            kp = np.where((ps >= 0.05) * (ps <= 0.95))
            temp_filtered_cohort = filtered_cohort.copy()
            filtered_cohort = filtered_cohort.iloc[kp]
            params = (ATE_A[feature][treatment_name], ATE_D[feature][treatment_name][time])
            summary = censor_bootstrap(X[kp], A.iloc[kp], y, D.iloc[kp], bootstrap_sample=100, other_word=y, params=params)
            filtered_cohort = temp_filtered_cohort.copy()
        else:
            summary = censor_bootstrap(X, A, y, D, bootstrap_sample=100, other_word=y, params=params)
        results.loc[len(results)] = [y] + summary
    
    results_df['t_efficacy'][feature] = results
    print("\n\n")

In [ ]:
results_df['adverse_events']['cohort_features']

In [ ]:
results_df['adverse_events']['omop_1_year_features']

In [ ]:
results_df['negative_controls']['cohort_features']

In [ ]:
results_df['negative_controls']['omop_1_year_features']

In [ ]:
results_df['t_efficacy']['cohort_features']

In [ ]:
results_df['t_efficacy']['omop_1_year_features']

In [ ]:
outcomes = ["adverse_events","t_efficacy"]
feats = ["cohort_features","omop_1_year_features"]

out_gname = {"adverse_events":"adverse_events","t_efficacy":"treatment efficacy"}
for o in outcomes:
    for f in feats:
        outcome_group_name = out_gname[o]
        results_df[o][f].to_pickle('Aggregate_results/'+f+'_'+treatment+'_'+outcome_group_name+'_'+'censor_v0'+'.pkl')
        results
        print("\n")

## Shapley Values

In [ ]:
lgbm = run_lgbm(feat_dict[final_feature][0],outcome_dict['treatment'][treatment_name],add_propensity=True,params=best_params,search=False)
cohort, clf = lgbm["cohort"],lgbm["classifier"]
df_coef = pd.DataFrame(list(zip(feat_dict[final_feature][1], clf.feature_importances_)), columns=['feature', 'lgbm importance'])
print(df_coef.shape)
display(df_coef.iloc[(-df_coef['lgbm importance'].abs()).argsort()].head(30).reset_index(drop=True))

In [ ]:
import shap
shap.initjs()

In [ ]:
best_A_params_cohort

In [ ]:
feat_dict['treatment']['cohort_features'][0].shape

In [ ]:
best_A_params_cohort[treatment_name]

In [ ]:
lgbm = run_lgbm(feat_dict['treatment']['cohort_features'][0], outcome_dict['treatment'][treatment_name], add_propensity = True, params = best_A_params_cohort[treatment_name], search = False) 
clf = lgbm[1]
clf

In [ ]:
%%time
lgbm = run_lgbm(feat_dict['treatment']['cohort_features'][0], outcome_dict['treatment'][treatment_name], add_propensity = True, params = best_A_params_cohort[treatment_name], search = False) 

cohort, clf = lgbm[0],lgbm[1]
explainer = shap.TreeExplainer(clf)
cols = feat_dict['treatment']['cohort_features'][1]
replace_list = [\
     ('age', 'Age'),\
     ('specialty_emergency/acute_group', 'Emergency medicine visit'),\
     ('specialty_advanced_specialist_group', 'Advanced specialist visit'),\
     ('alternatives_12_to_24_mo','Alternative antibiotics in 12 to 24 months'),\
     ('specialty_family_medicine_group', 'Family medicine visit'),\
     ('alternatives_6_to_12_mo','Alternative antibiotics in 6 to 12 months'),\
     ('specialty_internal_medicine_group', 'Internalist visit'),\
     ('specialty_OBGYN_group', 'Obstetricist/Gyencologist visit'),\
     ('specialty_other_group', 'Physician specialty other than prespecified groups*'),\
     ('alternatives_0_to_6_mo','Alternative antibiotics in 6 months'),\
     ('years_since_diagnosis', 'Prescription year'),\
     ('specialty_advanced_specialist_group', 'Advanced specialist visit'),\
     ('second_line_12_to_24_mo','Second-line antibiotics in 12 to 24 months'),\
     ('nitrofurantoin_12_to_24_mo','Nitrofurantoin in 12 to 24 months'),\
     ('specialty_internal_medicine_group', 'Internalist visit'),\
     ('second_line_6_to_12_mo','Second-line antibiotics in 6 to 12 months'),\
     ('urine_test_present', 'Urine test ordered'),\
     ('specialty_OBGYN_group', 'Obstetricist/Gyencologist visit'),\
     ('second_line_most_recent', 'Most recent antibiotic is second-line'),\
     ('specialty_emergency/acute_group', 'Emergency medicine visit'),\
     ('days_since_previous_uti', 'Days since previous UTI'),\
     ('previous_utis', 'UTI history within 2 years'),\
     ('specialty_emergency/acute_group', 'Emergency medicine visit'),\
     ('trimethoprim-sulfamethoxazole_most_recent', 'Most recent antibiotic is TMP-SMX'),\
     ('specialty_family_medicine_group', 'Family medicine visit'),\
     ('ph_urine_0', 'Urine pH value'),\
     ('creatinine_0', 'Creatinine value'),\
     ('neutrophil_p100_0', 'Neutrophil/100 leukocytes in blood value'),\
    ('diabetes_mellitus0_6_months','Diabetes mellitus in 6 months'),\
    ('hypertension0_6_months','Hypertension in 6 months'),\
     ('urine_test_present', 'Urine test ordered'),\
     ('years_since_diagnosis', 'Prescription year'),\
     ('treatment_','Alternative antibiotics prescribed')]
for i in replace_list:
     cols = [s.replace(i[0], i[1]) for s in cols]
X_df =  pd.DataFrame(feat_dict['treatment']['cohort_features'][0].todense(),\
     columns = cols)
shap_values = explainer.shap_values(X_df)
shap.summary_plot(shap_values, X_df, max_display = 10, class_names = ['First-line','Second-line'])

In [ ]:
%%time
lgbm = run_lgbm(feat_dict['treatment']['cohort_features'][0], outcome_dict['treatment'][treatment_name], add_propensity = True, params = best_A_params_cohort[treatment_name], search = False) 

cohort, clf = lgbm[0],lgbm[1]
explainer = shap.TreeExplainer(clf)
cols = feat_dict['treatment']['cohort_features'][1]
replace_list = [\
     ('age', 'Age'),\
     ('specialty_emergency/acute_group', 'Emergency medicine visit'),\
     ('specialty_advanced_specialist_group', 'Advanced specialist visit'),\
     ('alternatives_12_to_24_mo','Alternative antibiotics in 12 to 24 months'),\
     ('specialty_family_medicine_group', 'Family medicine visit'),\
     ('alternatives_6_to_12_mo','Alternative antibiotics in 6 to 12 months'),\
     ('specialty_internal_medicine_group', 'Internalist visit'),\
     ('specialty_OBGYN_group', 'Obstetricist/Gyencologist visit'),\
     ('specialty_other_group', 'Physician specialty other than prespecified groups*'),\
     ('alternatives_0_to_6_mo','Alternative antibiotics in 6 months'),\
     ('years_since_diagnosis', 'Prescription year'),\
     ('specialty_advanced_specialist_group', 'Advanced specialist visit'),\
     ('second_line_12_to_24_mo','Second-line antibiotics in 12 to 24 months'),\
     ('nitrofurantoin_12_to_24_mo','Nitrofurantoin in 12 to 24 months'),\
     ('specialty_internal_medicine_group', 'Internalist visit'),\
     ('second_line_6_to_12_mo','Second-line antibiotics in 6 to 12 months'),\
     ('urine_test_present', 'Urine test ordered'),\
     ('specialty_OBGYN_group', 'Obstetricist/Gyencologist visit'),\
     ('second_line_most_recent', 'Most recent antibiotic is second-line'),\
     ('specialty_emergency/acute_group', 'Emergency medicine visit'),\
     ('days_since_previous_uti', 'Days since previous UTI'),\
     ('previous_utis', 'UTI history within 2 years'),\
     ('specialty_emergency/acute_group', 'Emergency medicine visit'),\
     ('trimethoprim-sulfamethoxazole_most_recent', 'Most recent antibiotic is TMP-SMX'),\
     ('specialty_family_medicine_group', 'Family medicine visit'),\
     ('ph_urine_0', 'Urine pH value'),\
     ('creatinine_0', 'Creatinine value'),\
     ('neutrophil_p100_0', 'Neutrophil/100 leukocytes in blood value'),\
    ('diabetes_mellitus0_6_months','Diabetes mellitus in 6 months'),\
    ('hypertension0_6_months','Hypertension in 6 months'),\
     ('urine_test_present', 'Urine test ordered'),\
     ('years_since_diagnosis', 'Prescription year'),\
     ('treatment_','Alternative antibiotics prescribed')]
for i in replace_list:
     cols = [s.replace(i[0], i[1]) for s in cols]
X_df =  pd.DataFrame(feat_dict['treatment']['cohort_features'][0].todense(),\
     columns = cols)
shap_values = explainer.shap_values(X_df)
shap.summary_plot(shap_values[1],X_df, max_display = 10,  plot_type="dot", class_names = ['First-line','Second-line'])

In [ ]:
shap.summary_plot(shap_values, X_df, max_display = 10, class_names = ['First-line','Second-line'])

In [ ]:
%%time
lgbm = run_lgbm(feat_dict['censor']['cohort_features'][0], outcome_dict['censor']['30'], add_propensity = True, params = best_D_params_cohort[treatment_name]['30'], search = False) 

cohort, clf = lgbm[0],lgbm[1]
explainer = shap.TreeExplainer(clf)
cols = feat_dict['censor']['cohort_features'][1]
replace_list = [\
     ('days_since_previous_uti', 'Days since previous UTI'),\
     ('age', 'Age'),\
     ('previous_utis', 'UTI history with 2 years'),\
     ('specialty_emergency/acute_group', 'Emergency medicine visit'),\
     ('trimethoprim-sulfamethoxazole_most_recent', 'Most recent antibiotics is TMP-SMX'),\
     ('specialty_family_medicine_group', 'Family medicine visit'),\
     ('ph_urine_0', 'Urine pH value'),\
     ('urine_test_present', 'Urine test ordered'),\
     ('years_since_diagnosis', 'Prescription year'),\
     ('treatment_','Alternative antibiotics prescribed')]
for i in replace_list:
     cols = [s.replace(i[0], i[1]) for s in cols]
X_df =  pd.DataFrame(feat_dict['censor']['cohort_features'][0].todense(),\
     columns = cols)
shap_values = explainer.shap_values(X_df)
shap.summary_plot(shap_values, X_df, max_display = 10, class_names = ['Censored','Not censored'])

In [ ]:
%%time
lgbm = run_lgbm(feat_dict['treatment']['cohort_features'][0], outcome_dict['treatment'][treatment_name], add_propensity = True, params = best_A_params_cohort[treatment_name][', search = False) 

cohort, clf = lgbm[0],lgbm[1]
explainer = shap.TreeExplainer(clf)
cols = feat_dict['treatment']['cohort_features'][1]
replace_list = [\
     ('age', 'Age'),\
     ('specialty_emergency/acute_group', 'Emergency medicine visit'),\
     ('specialty_advanced_specialist_group', 'Advanced specialist visit'),\
     ('alternatives_12_to_24_mo','Alternative antibiotics in 12 to 24 months'),\
     ('specialty_family_medicine_group', 'Family medicine visit'),\
     ('alternatives_6_to_12_mo','Alternative antibiotics in 6 to 12 months'),\
     ('specialty_internal_medicine_group', 'Internalist visit'),\
     ('specialty_OBGYN_group', 'Obstetricist/Gyencologist visit'),\
     ('specialty_other_group', 'Physician specialty other than prespecified groups*'),\
     ('alternatives_0_to_6_mo','Alternative antibiotics in 6 months')]
for i in replace_list:
     cols = [s.replace(i[0], i[1]) for s in cols]
X_df =  pd.DataFrame(feat_dict['treatment']['cohort_features'][0].todense(),\
     columns = cols)
shap_values = explainer.shap_values(X_df)
shap.summary_plot(shap_values, X_df, max_display = 10, class_names = ['First-line','Alternatives'])

In [ ]:
cols[50:70]

### Calibration plot

In [ ]:
y_pred_column = 'propensity_score'
y_column = 'treatment'
condition_name = 'Second Line'


def calibration(cohort,y_pred,y_col,title=None,lim=None,cond_name=None,name=None,show=False):
    '''
    Input: cohort with a propensity column named propensity_score and antibiotic type column
           treatment name
    
    return plot'''
   
    bins = [np.quantile(cohort[y_pred], x/10) for x in range(11)]
    df_cali = pd.DataFrame(columns=['pred', 'actual', 'errorbar'])
    for i in range(10):
        temp = cohort[(cohort[y_pred]>=bins[i]) & (cohort[y_pred]<=bins[i+1])]
        x = np.mean(temp[y_pred])
        y = len(temp[temp[y_col]==1])/len(temp)
        std = np.std(temp[y_pred])
        df_cali.loc[len(df_cali)] = [x,y,1.96*std]

    fig = plt.figure(figsize=(7,7))
    # plt.plot(df_cali.pred, df_cali.actual, '-o')
    plt.errorbar(df_cali.pred, df_cali.actual, yerr=df_cali.errorbar, fmt='o', ls='--',color='black')
    plt.plot([0, 1], [0, 1], ls='--',color='black')
    plt.xlabel('Predicted Probability')
    plt.ylabel('Observed Probability')
    if lim is not None:
        plt.xlim(None,lim)
        plt.ylim(None,lim)
        
    if cond_name is None:
        plt.title('Calibration Plot for '+treatment)
    else:
        plt.title('Calibration Plot for '+cond_name)
    if name:
        plt.savefig(name)
    if not show:
        plt.close(fig)
    return fig


calibration(cohort,y_pred=y_pred_column,y_col=y_column,cond_name=condition_name,lim=0.85)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve
from scipy.stats import norm

# Sample data setup
# df = pd.DataFrame({'y_pred': [0.1, 0.4, 0.35, 0.8, 0.7, 0.2, 0.5, 0.6, 0.9, 0.75], 'y': [0, 0, 1, 1, 1, 0, 1, 1, 1, 0]})

# Generate the calibration data
true_prob, pred_prob = calibration_curve(df['y'], df['y_pred'], n_bins=10, strategy='uniform')

# Calculate bin sizes
bin_sizes = np.histogram(df['y_pred'], bins=10, range=(0, 1))[0]

# Calculate standard error for the binomial proportion in each bin
se = np.sqrt(true_prob * (1 - true_prob) / bin_sizes)

# Calculate the z-value for the 95% confidence interval
z = norm.ppf(0.975)  # Two-tailed, hence 0.975 instead of 0.95

# Calculate the confidence intervals
ci_lower = true_prob - z * se
ci_upper = true_prob + z * se

# Plotting the calibration curve with error bars
plt.figure(figsize=(8, 6))
plt.errorbar(pred_prob, true_prob, yerr=z * se, fmt='o-', color='b', label='Calibration plot w/ 95% CI', linewidth=1, markersize=5)
plt.plot([0, 1], [0, 1], linestyle='--', color='red', label='Perfectly calibrated')
plt.xlabel('Mean predicted probability')
plt.ylabel('Fraction of positives')
plt.title('Calibration Plot with 95% Confidence Intervals')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
bin_sizes

In [ ]:
true_prob

In [ ]:
np.argwhere(bin_sizes==0)

## Calibration Plots

### Second Line

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve
from scipy.stats import norm
y_pred_column = 'propensity_score'
y_column = 'treatment'
condition_name = 'Second Line'

def get_se(true_prob,bin_sizes):
    updated_true_prob = np.insert(true_prob,np.argwhere(bin_sizes==0)[0],np.nan)
    se = np.sqrt(updated_true_prob * (1 - updated_true_prob) / bin_sizes)
    se = se[~np.isnan(se)]
    return se

_,_,_, T_Test = train_test_split(cohort, cohort[y_column], test_size = 0.2, random_state = 0)
reduced_cohort = cohort.iloc[T_Test.index]

# Generate the calibration data
true_prob, pred_prob = calibration_curve(reduced_cohort[y_column], reduced_cohort[y_pred_column], n_bins=10, strategy='uniform')

# Calculate bin sizes
bin_sizes = np.histogram(reduced_cohort[y_pred_column], bins=10, range=(0, 1))[0]

# Calculate standard error for the binomial proportion in each bin
se = get_se(true_prob,bin_sizes)

# Calculate the z-value for the 95% confidence interval
z = norm.ppf(0.975)  # Two-tailed, hence 0.975 instead of 0.95

# Calculate the confidence intervals
ci_lower = true_prob - z * se
upper = (true_prob + z * se)
upper[np.argwhere(true_prob+z*se > 1)] = 1
y_err = [z*se,(upper -true_prob)]
# Plotting the calibration curve with error bars
plt.figure(figsize=(8, 6))
plt.errorbar(pred_prob, true_prob, yerr=y_err,fmt='o--', color='b', label='Calibration curve & 95% Confidence Interval')
# plt.fill_between(pred_prob, (true_prob - z*se), upper, color='b', alpha=.1)
plt.plot([0, 1], [0, 1], linestyle='--', color='red', label='Perfectly calibrated')
plt.xlabel('Predicted probability',fontsize=14)
plt.ylabel('Observed Probability',fontsize=14)
plt.title(f'Calibration Plot for {condition_name}',fontsize=14)
plt.legend(fontsize=11)
plt.show()

In [ ]:
best_A_params_cohort[treatment_name]

In [ ]:
best_D_params_cohort[treatment_name]

### Second-line

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve
from scipy.stats import norm
y_pred_column = 'propensity_score'
y_column = 'treatment'

lgbm = run_lgbm(feat_dict['treatment']['cohort_features'][0], outcome_dict['treatment'][treatment_name], add_propensity = True, params = best_A_params_cohort[treatment_name], search = False) 

cohort, clf = lgbm[0],lgbm[1]
print(f"Test AUROC is: {lgbm[4][1]}")
def get_se(true_prob,bin_sizes):
    
    updated_true_prob = true_prob
    for idx in np.where(bin_sizes == 0)[0]:
        updated_true_prob = np.insert(updated_true_prob, idx, np.nan)
    se = np.sqrt(updated_true_prob * (1 - updated_true_prob) / bin_sizes)
    se = se[~np.isnan(se)]
    return se

_,_,_, T_Test = train_test_split(cohort, cohort[y_column], test_size = 0.2, random_state = 0)
reduced_cohort = cohort.iloc[T_Test.index]

# Generate the calibration data
true_prob, pred_prob = calibration_curve(reduced_cohort[y_column], reduced_cohort[y_pred_column], n_bins=10, strategy='uniform')

# Calculate bin sizes
bin_sizes = np.histogram(reduced_cohort[y_pred_column], bins=10, range=(0, 1))[0]

# Calculate standard error for the binomial proportion in each bin
se = get_se(true_prob,bin_sizes)

# Calculate the z-value for the 95% confidence interval
z = norm.ppf(0.975)  # Two-tailed, hence 0.975 instead of 0.95

# Calculate the confidence intervals
ci_lower = true_prob - z * se
upper = (true_prob + z * se)
upper[np.argwhere(true_prob+z*se > 1)] = 1
y_err = [z*se,(upper -true_prob)]
# Plotting the calibration curve with error bars
plt.figure(figsize=(8, 6))
plt.errorbar(pred_prob, true_prob, yerr=y_err,fmt='o--', color='black', label='Calibration curve & 95% Confidence Interval')
# plt.fill_between(pred_prob, (true_prob - z*se), upper, color='b', alpha=.1)
plt.plot([0, 1], [0, 1], linestyle='--', color='red', label='Perfect calibration')
plt.xlabel('Predicted probability',fontsize=14)
plt.ylabel('Observed Probability',fontsize=14)
plt.title(f'Calibration Plot for Second-line Propensity Model',fontsize=14)
plt.legend(fontsize=11)
plt.show()

### Censor 15

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve
from scipy.stats import norm
y_pred_column = 'propensity_score'
y_column = 'treatment'
condition_name = '15-Day Censoring'

lgbm = run_lgbm(feat_dict['censor']['cohort_features'][0], outcome_dict['censor']['15'], add_propensity = True, params = best_D_params_cohort[treatment_name]['15'], search = False) 

cohort, clf = lgbm[0],lgbm[1]
print(f"Test AUROC is: {lgbm[4][1]}")
def get_se(true_prob,bin_sizes):
    
    updated_true_prob = true_prob
    for idx in np.where(bin_sizes == 0)[0]:
        updated_true_prob = np.insert(updated_true_prob, idx, np.nan)
    se = np.sqrt(updated_true_prob * (1 - updated_true_prob) / bin_sizes)
    se = se[~np.isnan(se)]
    return se

_,_,_, T_Test = train_test_split(cohort, cohort[y_column], test_size = 0.2, random_state = 0)
reduced_cohort = cohort.iloc[T_Test.index]

# Generate the calibration data
true_prob, pred_prob = calibration_curve(reduced_cohort[y_column], reduced_cohort[y_pred_column], n_bins=10, strategy='uniform')

# Calculate bin sizes
bin_sizes = np.histogram(reduced_cohort[y_pred_column], bins=10, range=(0, 1))[0]

# Calculate standard error for the binomial proportion in each bin
se = get_se(true_prob,bin_sizes)

# Calculate the z-value for the 95% confidence interval
z = norm.ppf(0.975)  # Two-tailed, hence 0.975 instead of 0.95

# Calculate the confidence intervals
ci_lower = true_prob - z * se
upper = (true_prob + z * se)
upper[np.argwhere(true_prob+z*se > 1)] = 1
y_err = [z*se,(upper -true_prob)]
# Plotting the calibration curve with error bars
plt.figure(figsize=(8, 6))
plt.errorbar(pred_prob, true_prob, yerr=y_err,fmt='o--', color='black', label='Calibration curve & 95% Confidence Interval')
# plt.fill_between(pred_prob, (true_prob - z*se), upper, color='b', alpha=.1)
plt.plot([0, 1], [0, 1], linestyle='--', color='red', label='Perfect calibration')
plt.xlabel('Predicted probability',fontsize=14)
plt.ylabel('Observed Probability',fontsize=14)
plt.title(f'Calibration Plot for Second-line {condition_name} Model',fontsize=14)
plt.legend(fontsize=11)

plt.show()

### Censor 30

In [ ]:
outcome_dict['censor']['30'].value_counts()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve
from scipy.stats import norm
y_pred_column = 'propensity_score'
y_column = 'treatment'
condition_name = '30-Day Censoring'

logreg = run_logreg(feat_dict['censor']['cohort_features'][0], outcome_dict['censor']['30'], add_propensity = True, params = best_D_params_cohort[treatment_name]['30'], search = False) 

cohort, clf = logreg[0],logreg[1]
print(f"Test AUROC is: {logreg[4][1]}")
def get_se(true_prob,bin_sizes):
    
    updated_true_prob = true_prob
    for idx in np.where(bin_sizes == 0)[0]:
        updated_true_prob = np.insert(updated_true_prob, idx, np.nan)
    se = np.sqrt(updated_true_prob * (1 - updated_true_prob) / bin_sizes)
    se = se[~np.isnan(se)]
    return se

_,_,_, T_Test = train_test_split(cohort, cohort[y_column], test_size = 0.2, random_state = 0)
reduced_cohort = cohort.iloc[T_Test.index]

# Generate the calibration data
true_prob, pred_prob = calibration_curve(reduced_cohort[y_column], reduced_cohort[y_pred_column], n_bins=10, strategy='uniform')

# Calculate bin sizes
bin_sizes = np.histogram(reduced_cohort[y_pred_column], bins=10, range=(0, 1))[0]

# Calculate standard error for the binomial proportion in each bin
se = get_se(true_prob,bin_sizes)

# Calculate the z-value for the 95% confidence interval
z = norm.ppf(0.975)  # Two-tailed, hence 0.975 instead of 0.95

# Calculate the confidence intervals
ci_lower = true_prob - z * se
upper = (true_prob + z * se)
upper[np.argwhere(true_prob+z*se > 1)] = 1
y_err = [z*se,(upper -true_prob)]
# Plotting the calibration curve with error bars
plt.figure(figsize=(8, 6))
plt.errorbar(pred_prob, true_prob, yerr=y_err,fmt='o--', color='black', label='Calibration curve & 95% Confidence Interval')
# plt.fill_between(pred_prob, (true_prob - z*se), upper, color='b', alpha=.1)
plt.plot([0, 1], [0, 1], linestyle='--', color='red', label='Perfect calibration')
plt.xlabel('Predicted probability',fontsize=14)
plt.ylabel('Observed Probability',fontsize=14)
plt.title(f'Calibration Plot for Second-line {condition_name} Model',fontsize=14)

plt.legend(fontsize=11)
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve
from scipy.stats import norm
y_pred_column = 'propensity_score'
y_column = 'treatment'
condition_name = '90-Day Censoring'

lgbm = run_lgbm(feat_dict['censor']['cohort_features'][0], outcome_dict['censor']['90'], add_propensity = True, params = best_D_params_cohort[treatment_name]['90'], search = False) 

cohort, clf = lgbm[0],lgbm[1]
print(f"Test AUROC is: {lgbm[4][1]}")
def get_se(true_prob,bin_sizes):
    
    updated_true_prob = true_prob
    for idx in np.where(bin_sizes == 0)[0]:
        updated_true_prob = np.insert(updated_true_prob, idx, np.nan)
    se = np.sqrt(updated_true_prob * (1 - updated_true_prob) / bin_sizes)
    se = se[~np.isnan(se)]
    return se

_,_,_, T_Test = train_test_split(cohort, cohort[y_column], test_size = 0.2, random_state = 0)
reduced_cohort = cohort.iloc[T_Test.index]

# Generate the calibration data
true_prob, pred_prob = calibration_curve(reduced_cohort[y_column], reduced_cohort[y_pred_column], n_bins=10, strategy='uniform')

# Calculate bin sizes
bin_sizes = np.histogram(reduced_cohort[y_pred_column], bins=10, range=(0, 1))[0]

# Calculate standard error for the binomial proportion in each bin
se = get_se(true_prob,bin_sizes)

# Calculate the z-value for the 95% confidence interval
z = norm.ppf(0.975)  # Two-tailed, hence 0.975 instead of 0.95

# Calculate the confidence intervals
ci_lower = true_prob - z * se
upper = (true_prob + z * se)
upper[np.argwhere(true_prob+z*se > 1)] = 1
y_err = [z*se,(upper -true_prob)]
# Plotting the calibration curve with error bars
plt.figure(figsize=(8, 6))
plt.errorbar(pred_prob, true_prob, yerr=y_err,fmt='o--', color='black', label='Calibration curve & 95% Confidence Interval')
# plt.fill_between(pred_prob, (true_prob - z*se), upper, color='b', alpha=.1)
plt.plot([0, 1], [0, 1], linestyle='--', color='red', label='Perfect calibration')
plt.xlabel('Predicted probability',fontsize=14)
plt.ylabel('Observed Probability',fontsize=14)
plt.title(f'Calibration Plot for Second-line {condition_name} Model',fontsize=14)

plt.legend(fontsize=11)
plt.show()